In [ ]:
import polars as pl
import xgboost as xgb
import numpy as np
import os
import gc
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


MASTER_FILE = "draft_oracle_master_data.parquet"
FEATURE_FILE = "draft_oracle_feature_store.parquet"
TRAIN_SET_FILE = "draft_oracle_training_set.parquet"
MODEL_FILE = "draft_oracle_brain.json"

In [ ]:
def enrich_combat_logic(df):
    print("Calculando Lógica de Combate (Damage Profile & Counters)...")

    sides = ["blue", "red"]

    for side in sides:

        magic_cols = [c for c in df.columns if f"{side}_" in c and ("stat_magic_dmg" in c or "avg_magic_dmg" in c)]
        phys_cols  = [c for c in df.columns if f"{side}_" in c and ("stat_phys_dmg" in c or "avg_phys_dmg" in c)]

        true_cols  = [c for c in df.columns if f"{side}_" in c and ("stat_true_dmg" in c or "avg_true_dmg" in c)]


        tank_cols  = [c for c in df.columns if f"{side}_" in c and "stat_mitigated" in c]
        heal_cols  = [c for c in df.columns if f"{side}_" in c and "stat_heal" in c]
        cc_cols    = [c for c in df.columns if f"{side}_" in c and "stat_hard_cc" in c]


        total_magic = pl.sum_horizontal(magic_cols) if magic_cols else pl.lit(0)
        total_phys  = pl.sum_horizontal(phys_cols) if phys_cols else pl.lit(0)
        total_true  = pl.sum_horizontal(true_cols) if true_cols else pl.lit(0)
        total_tank  = pl.sum_horizontal(tank_cols) if tank_cols else pl.lit(0)
        total_heal  = pl.sum_horizontal(heal_cols) if heal_cols else pl.lit(0)
        total_cc    = pl.sum_horizontal(cc_cols) if cc_cols else pl.lit(0)


        df = df.with_columns([
            total_magic.fill_null(0).alias(f"{side}_total_magic_dmg"),
            total_phys.fill_null(0).alias(f"{side}_total_phys_dmg"),
            total_true.fill_null(0).alias(f"{side}_total_true_dmg"),
            total_tank.fill_null(0).alias(f"{side}_total_tankiness"),
            total_heal.fill_null(0).alias(f"{side}_total_sustain"),
            total_cc.fill_null(0).alias(f"{side}_total_cc")
        ])


        total_dmg = pl.col(f"{side}_total_magic_dmg") + pl.col(f"{side}_total_phys_dmg") + pl.col(f"{side}_total_true_dmg") + 1
        df = df.with_columns(
            (pl.col(f"{side}_total_magic_dmg") / total_dmg).alias(f"{side}_magic_dmg_ratio")
        )


    df = df.with_columns([

        ((pl.col("blue_total_magic_dmg") + pl.col("blue_total_true_dmg")) / (pl.col("red_total_tankiness") + 1)).alias("blue_shred_efficiency"),
        ((pl.col("red_total_magic_dmg") + pl.col("red_total_true_dmg")) / (pl.col("blue_total_tankiness") + 1)).alias("red_shred_efficiency"),


        ((pl.col("blue_total_phys_dmg") + pl.col("blue_total_magic_dmg")) / (pl.col("red_total_sustain") + 1000)).alias("blue_anti_sustain_burst"),
        ((pl.col("red_total_phys_dmg") + pl.col("red_total_magic_dmg")) / (pl.col("blue_total_sustain") + 1000)).alias("red_anti_sustain_burst")
    ])

    blue_volatility = sum([pl.col(f"blue_{role}_var_gold_volatility").fill_null(0) for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]])
    red_volatility  = sum([pl.col(f"red_{role}_var_gold_volatility").fill_null(0)  for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]])

    df = df.with_columns(
        (blue_volatility - red_volatility).alias("diff_team_volatility")
    )

    return df

In [ ]:
def assemble_and_clean_dataset_ultimate():
    print("REGENERANDO DATASET V4.2 (Incluyendo Risk & Volatility)...")


    df_features = pl.read_parquet(FEATURE_FILE)


    master_full = pl.read_parquet(MASTER_FILE)
    df_matches = master_full.select([
        "game_id", "region", "patch", "duration", "target",
        "champ_id", "position", "side"
    ])
    del master_full
    gc.collect()

    print("   Pegando Stats + Estilos + Riesgo...")
    enriched = df_matches.join(
        df_features,
        on=["champ_id", "position", "region"],
        how="left"
    )

    print("   Pivotando a formato Wide...")

    enriched = enriched.with_columns([
        pl.when(pl.col("side") == 100).then(pl.lit("blue")).otherwise(pl.lit("red")).alias("side_str")
    ])
    enriched = enriched.with_columns(
        (pl.col("side_str") + "_" + pl.col("position")).alias("pivot_col")
    )


    cols_to_pivot = [
        c for c in df_features.columns
        if c.startswith("stat_")
        or c.startswith("avg_")
        or c.startswith("z_style_")
        or c.startswith("var_")
    ]
    cols_to_pivot.append("champ_id")

    wide_df = enriched.pivot(
        values=cols_to_pivot,
        index=["game_id", "target", "region", "patch"],
        columns="pivot_col",
        aggregate_function="first"
    )

    print("   Normalizando nombres...")
    new_names = {}
    for col in wide_df.columns:
        if "blue" in col or "red" in col:
            parts = col.split("_")
            if "blue" in parts:
                idx = parts.index("blue")
                team_pos = f"{parts[idx]}_{parts[idx+1]}"
                stat_parts = [p for i, p in enumerate(parts) if i != idx and i != idx+1]
                stat_name = "_".join(stat_parts)
                new_names[col] = f"{team_pos}_{stat_name}"
            elif "red" in parts:
                idx = parts.index("red")
                team_pos = f"{parts[idx]}_{parts[idx+1]}"
                stat_parts = [p for i, p in enumerate(parts) if i != idx and i != idx+1]
                stat_name = "_".join(stat_parts)
                new_names[col] = f"{team_pos}_{stat_name}"

    wide_df = wide_df.rename(new_names)

    print(f"   Dataset V4.2 Guardado: {wide_df.shape}")
    wide_df.write_parquet(TRAIN_SET_FILE)
    return wide_df


df = assemble_and_clean_dataset_ultimate()

In [ ]:
def build_interaction_matrices(df_master):
    print("Construyendo Cerebro Táctico (Sinergias y Counters)...")


    matches = df_master.lazy().select([
        "game_id", "side", "champ_id", "position",
        pl.col("target").alias("win")
    ])

    print("   -> Calculando pares de aliados ganadores...")
    synergy = (
        matches.join(matches, on=["game_id", "side"])
        .filter(pl.col("champ_id") != pl.col("champ_id_right"))
        .group_by(["champ_id", "champ_id_right"])
        .agg([
            pl.col("win").mean().alias("syn_winrate"),
            pl.count().alias("matches")
        ])
        .filter(pl.col("matches") > 50)
        .collect()
    )

    print("   -> Calculando matchups directos...")
    counters = (
        matches.join(matches, on="game_id")
        .filter(pl.col("side") != pl.col("side_right"))
        .filter(pl.col("position") == pl.col("position_right"))
        .group_by(["champ_id", "champ_id_right"])
        .agg([
            pl.col("win").mean().alias("counter_winrate"),
            pl.count().alias("matches")
        ])
        .filter(pl.col("matches") > 50)
        .collect()
    )

    return synergy, counters

def assemble_and_clean_dataset_ultimate():
    print("REGENERANDO DATASET V4.3 (FIX DEFINITIVO)...")


    df_features = pl.read_parquet(FEATURE_FILE)


    master_full = pl.read_parquet(MASTER_FILE)
    df_matches = master_full.select([
        "game_id", "region", "patch", "duration", "target",
        "champ_id", "position", "side"
    ])
    del master_full
    gc.collect()

    print("   Pegando Todo el ADN...")
    enriched = df_matches.join(
        df_features,
        on=["champ_id", "position", "region"],
        how="left"
    )

    print("   Pivotando a formato Wide...")

    enriched = enriched.with_columns([
        pl.when(pl.col("side") == 100).then(pl.lit("blue")).otherwise(pl.lit("red")).alias("side_str")
    ])
    enriched = enriched.with_columns(
        (pl.col("side_str") + "_" + pl.col("position")).alias("pivot_col")
    )


    cols_to_pivot = [
        c for c in df_features.columns
        if c.startswith("stat_")
        or c.startswith("avg_")
        or c.startswith("z_style_")
        or c.startswith("var_")
    ]
    cols_to_pivot.append("champ_id")

    wide_df = enriched.pivot(
        values=cols_to_pivot,
        index=["game_id", "target", "region", "patch"],
        columns="pivot_col",
        aggregate_function="first"
    )

    print("   Normalizando nombres...")
    new_names = {}
    for col in wide_df.columns:
        if "blue" in col or "red" in col:
            parts = col.split("_")
            if "blue" in parts:
                idx = parts.index("blue")
                team_pos = f"{parts[idx]}_{parts[idx+1]}"
                stat_parts = [p for i, p in enumerate(parts) if i != idx and i != idx+1]
                stat_name = "_".join(stat_parts)
                new_names[col] = f"{team_pos}_{stat_name}"
            elif "red" in parts:
                idx = parts.index("red")
                team_pos = f"{parts[idx]}_{parts[idx+1]}"
                stat_parts = [p for i, p in enumerate(parts) if i != idx and i != idx+1]
                stat_name = "_".join(stat_parts)
                new_names[col] = f"{team_pos}_{stat_name}"

    wide_df = wide_df.rename(new_names)

    print(f"   Dataset V4.3 Guardado: {wide_df.shape}")
    wide_df.write_parquet(TRAIN_SET_FILE)
    return wide_df


df = assemble_and_clean_dataset_ultimate()

In [ ]:
def enrich_critical_synergies(df):
    print("Tejiendo la Red de Sinergias Críticas (Graph Theory)...")


    try:
        synergy_matrix = pl.read_parquet("draft_oracle_synergy_matrix.parquet")
    except:
        print("No se encontró la matriz de sinergia. Saltando paso de grafos.")
        return df


    critical_pairs = [
        ("MIDDLE", "JUNGLE", "syn_mid_jg"),
        ("BOTTOM", "UTILITY", "syn_bot_duo"),
        ("TOP", "JUNGLE", "syn_top_jg"),
    ]

    sides = ["blue", "red"]

    for side in sides:
        for role1, role2, feat_name in critical_pairs:
            col_id1 = f"{side}_{role1}_champ_id"
            col_id2 = f"{side}_{role2}_champ_id"


            if col_id1 not in df.columns or col_id2 not in df.columns:
                continue

            temp_syn = synergy_matrix.select([
                pl.col("champ_id").alias(col_id1),
                pl.col("champ_id_right").alias(col_id2),
                pl.col("syn_winrate").alias(f"{side}_{feat_name}")
            ])

            df = df.join(temp_syn, on=[col_id1, col_id2], how="left")
            df = df.with_columns(pl.col(f"{side}_{feat_name}").fill_null(0.5))


    if "blue_syn_bot_duo" in df.columns and "red_syn_bot_duo" in df.columns:
        df = df.with_columns((pl.col("blue_syn_bot_duo") - pl.col("red_syn_bot_duo")).alias("gap_syn_bot_duo"))
    if "blue_syn_mid_jg" in df.columns and "red_syn_mid_jg" in df.columns:
        df = df.with_columns((pl.col("blue_syn_mid_jg") - pl.col("red_syn_mid_jg")).alias("gap_syn_mid_jg"))

    return df


In [ ]:

def load_and_prep_data():
    import os
    if os.path.exists(TRAIN_SET_FILE):
        print(f"Cargando dataset pre-cocinado: {TRAIN_SET_FILE}")
        df = pl.read_parquet(TRAIN_SET_FILE)
    else:

        raise FileNotFoundError("Genera primero el training_set con el código anterior.")

    print("Aplicando Lógica de Head Coach (Archetypes & Synergies)...")


    sides = ["blue", "red"]
    new_cols = []

    for side in sides:
        enemy_side = "red" if side == "blue" else "blue"


        dmg_cols = [c for c in df.columns if f"{side}_" in c and "stat_dpm" in c]
        mitigated_cols = [c for c in df.columns if f"{side}_" in c and "stat_mitigated" in c]
        heal_cols = [c for c in df.columns if f"{side}_" in c and "stat_heal" in c]


        cc_cols = [c for c in df.columns if f"{side}_" in c and "stat_hard_cc" in c]
        vision_cols = [c for c in df.columns if f"{side}_" in c and "stat_vision" in c]


        mid_roam = f"{side}_middle_stat_roam_kills"
        jg_roam = f"{side}_jungle_stat_roam_kills"


        team_cc = pl.sum_horizontal(cc_cols)
        team_tankiness = pl.sum_horizontal(mitigated_cols)
        engage_score = ((team_cc * team_tankiness) / 10000).alias(f"{side}_archetype_engage")
        new_cols.append(engage_score)


        sustain_score = (pl.sum_horizontal(heal_cols) + team_tankiness).alias(f"{side}_archetype_sustain")
        new_cols.append(sustain_score)


        vision_score = pl.sum_horizontal(vision_cols)
        pick_potential = (vision_score * team_cc).alias(f"{side}_archetype_pick")
        new_cols.append(pick_potential)


        if mid_roam in df.columns and jg_roam in df.columns:
            mid_jg_syn = (pl.col(mid_roam) + pl.col(jg_roam)).alias(f"{side}_syn_mid_jungle")
            new_cols.append(mid_jg_syn)

        prio_cols = [c for c in df.columns if f"{side}_" in c and "stat_lane_priority_score" in c]
        rel_cols = [c for c in df.columns if f"{side}_" in c and "stat_reliability_index" in c]
        early_cols = [c for c in df.columns if f"{side}_" in c and "stat_wr_early" in c]
        late_cols = [c for c in df.columns if f"{side}_" in c and "stat_wr_late" in c]


        if len(prio_cols) > 0:

            new_cols.append(pl.sum_horizontal(prio_cols).alias(f"{side}_strat_total_priority"))


            new_cols.append(pl.mean_horizontal(rel_cols).alias(f"{side}_strat_avg_reliability"))


            new_cols.append(pl.mean_horizontal(early_cols).alias(f"{side}_strat_early_power"))
            new_cols.append(pl.mean_horizontal(late_cols).alias(f"{side}_strat_late_power"))


    blue_cc = pl.sum_horizontal([c for c in df.columns if "blue_" in c and "stat_hard_cc" in c])
    red_cc = pl.sum_horizontal([c for c in df.columns if "red_" in c and "stat_hard_cc" in c])

    blue_range = pl.sum_horizontal([c for c in df.columns if "blue_" in c and "stat_dpm" in c])
    red_mitigation = pl.sum_horizontal([c for c in df.columns if "red_" in c and "stat_mitigated" in c])


    new_cols.append((blue_cc - red_cc).alias("diff_cc_gap"))


    new_cols.append((blue_range / (red_mitigation + 1)).alias("diff_shred_potential"))


    df = df.with_columns(new_cols)
    return df


if 'load_and_prep_data' not in globals(): raise Exception("Ejecuta load_and_prep_data()")
print("Cargando dataset...")
df = load_and_prep_data()


def enrich_combat_logic(df):
    print("[1/3] Calculando Lógica de Combate (Daño vs Resistencia)...")
    sides = ["blue", "red"]
    for side in sides:

        magic_cols = [c for c in df.columns if f"{side}_" in c and ("stat_magic_dmg" in c or "avg_magic_dmg" in c)]
        phys_cols  = [c for c in df.columns if f"{side}_" in c and ("stat_phys_dmg" in c or "avg_phys_dmg" in c)]
        true_cols  = [c for c in df.columns if f"{side}_" in c and ("stat_true_dmg" in c or "avg_true_dmg" in c)]
        tank_cols  = [c for c in df.columns if f"{side}_" in c and "stat_mitigated" in c]
        heal_cols  = [c for c in df.columns if f"{side}_" in c and "stat_heal" in c]
        cc_cols    = [c for c in df.columns if f"{side}_" in c and "stat_hard_cc" in c]


        total_magic = pl.sum_horizontal(magic_cols) if magic_cols else pl.lit(0)
        total_phys  = pl.sum_horizontal(phys_cols) if phys_cols else pl.lit(0)
        total_true  = pl.sum_horizontal(true_cols) if true_cols else pl.lit(0)
        total_tank  = pl.sum_horizontal(tank_cols) if tank_cols else pl.lit(0)
        total_heal  = pl.sum_horizontal(heal_cols) if heal_cols else pl.lit(0)
        total_cc    = pl.sum_horizontal(cc_cols) if cc_cols else pl.lit(0)


        df = df.with_columns([
            total_magic.fill_null(0).alias(f"{side}_total_magic_dmg"),
            total_phys.fill_null(0).alias(f"{side}_total_phys_dmg"),
            total_true.fill_null(0).alias(f"{side}_total_true_dmg"),
            total_tank.fill_null(0).alias(f"{side}_total_tankiness"),
            total_heal.fill_null(0).alias(f"{side}_total_sustain"),
            total_cc.fill_null(0).alias(f"{side}_total_cc")
        ])


        total_dmg = pl.col(f"{side}_total_magic_dmg") + pl.col(f"{side}_total_phys_dmg") + pl.col(f"{side}_total_true_dmg") + 1
        df = df.with_columns((pl.col(f"{side}_total_magic_dmg") / total_dmg).alias(f"{side}_magic_dmg_ratio"))

    df = df.with_columns([
        ((pl.col("blue_total_magic_dmg") + pl.col("blue_total_true_dmg")) / (pl.col("red_total_tankiness") + 1)).alias("blue_shred_efficiency"),
        ((pl.col("red_total_magic_dmg") + pl.col("red_total_true_dmg")) / (pl.col("blue_total_tankiness") + 1)).alias("red_shred_efficiency"),
        ((pl.col("blue_total_phys_dmg") + pl.col("blue_total_magic_dmg")) / (pl.col("red_total_sustain") + 1000)).alias("blue_anti_sustain_burst"),
        ((pl.col("red_total_phys_dmg") + pl.col("red_total_magic_dmg")) / (pl.col("blue_total_sustain") + 1000)).alias("red_anti_sustain_burst")
    ])
    return df

def enrich_behavioral_strategy(df):
    print("[2/3] Analizando Psicología, Economía y RIESGO (V12)...")
    sides = ["blue", "red"]
    for side in sides:

        jg_aggro = pl.col(f"{side}_JUNGLE_z_style_gank_heaviness").fill_null(0)
        lanes_aggro = (pl.col(f"{side}_TOP_z_style_lane_dominance").fill_null(0) +
                       pl.col(f"{side}_MIDDLE_z_style_lane_dominance").fill_null(0) +
                       pl.col(f"{side}_BOTTOM_z_style_lane_dominance").fill_null(0)) / 3
        df = df.with_columns((jg_aggro * lanes_aggro).alias(f"{side}_strat_gank_compatibility"))


        gold_hunger = sum([pl.col(f"{side}_{role}_z_style_gold_hunger").fill_null(0)
                           for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]])
        df = df.with_columns(gold_hunger.alias(f"{side}_strat_resource_friction"))


        invade_potential = pl.col(f"{side}_JUNGLE_z_style_invade_pressure").fill_null(0)
        backup_potential = (pl.col(f"{side}_TOP_z_style_roaming_tendency").fill_null(0) + pl.col(f"{side}_MIDDLE_z_style_roaming_tendency").fill_null(0))
        df = df.with_columns((invade_potential * backup_potential).alias(f"{side}_strat_invade_safety"))


    try:
        blue_vol = sum([pl.col(f"blue_{role}_var_gold_volatility").fill_null(0) for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]])
        red_vol  = sum([pl.col(f"red_{role}_var_gold_volatility").fill_null(0)  for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]])
        df = df.with_columns((blue_vol - red_vol).alias("diff_team_volatility"))
        print("   Variable de Volatilidad (Riesgo) Calculada.")
    except:
        print("   No se encontraron columnas de Volatilidad. Se entrena sin factor de riesgo.")

    return df

def enrich_critical_synergies(df):
    print("[3/3] Tejiendo Grafos de Sinergia (Mid-Jungle Links)...")
    try:
        synergy_matrix = pl.read_parquet("draft_oracle_synergy_matrix.parquet")
    except:
        print("   No se encontró la matriz de sinergia.")
        return df

    critical_pairs = [("MIDDLE", "JUNGLE", "syn_mid_jg"), ("BOTTOM", "UTILITY", "syn_bot_duo"), ("TOP", "JUNGLE", "syn_top_jg")]
    sides = ["blue", "red"]

    for side in sides:
        for role1, role2, feat_name in critical_pairs:
            col_id1 = f"{side}_{role1}_champ_id"
            col_id2 = f"{side}_{role2}_champ_id"
            if col_id1 not in df.columns or col_id2 not in df.columns: continue

            temp_syn = synergy_matrix.select([
                pl.col("champ_id").alias(col_id1), pl.col("champ_id_right").alias(col_id2), pl.col("syn_winrate").alias(f"{side}_{feat_name}")
            ])
            df = df.join(temp_syn, on=[col_id1, col_id2], how="left")
            df = df.with_columns(pl.col(f"{side}_{feat_name}").fill_null(0.5))

    if "blue_syn_mid_jg" in df.columns and "red_syn_mid_jg" in df.columns:
        df = df.with_columns((pl.col("blue_syn_mid_jg") - pl.col("red_syn_mid_jg")).alias("gap_syn_mid_jg"))
    return df


TRAIN_FILE = "draft_oracle_training_set.parquet"
print(f"Cargando Dataset desde {TRAIN_FILE}...")

if not os.path.exists(TRAIN_FILE):
    raise FileNotFoundError("Ejecuta primero el Assembler.")


df = pl.read_parquet(TRAIN_FILE)
float_cols = [c for c in df.columns if df[c].dtype == pl.Float64]
if float_cols:
    print("Comprimiendo datos a Float32...")
    df = df.with_columns([pl.col(c).cast(pl.Float32) for c in float_cols])


if 'enrich_combat_logic' in globals(): df = enrich_combat_logic(df)
if 'enrich_behavioral_strategy' in globals(): df = enrich_behavioral_strategy(df)
if 'enrich_critical_synergies' in globals(): df = enrich_critical_synergies(df)


roles = ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]
duel_stats = ["stat_winrate", "stat_gpm", "stat_dpm", "z_style_lane_dominance"]
new_cols = []
for role in roles:
    for stat in duel_stats:
        col_blue, col_red = f"blue_{role}_{stat}", f"red_{role}_{stat}"
        if col_blue in df.columns and col_red in df.columns:
            new_cols.append((pl.col(col_blue) - pl.col(col_red)).alias(f"duel_{role}_{stat}"))
if new_cols: df = df.with_columns(new_cols)


feature_cols = [c for c in df.columns if (
    "stat_" in c or "rune_" in c or "archetype_" in c or "gap_" in c or "duel_" in c or
    "_total_" in c or "_ratio" in c or "_efficiency" in c or "_burst" in c or
    "z_style_" in c or "_strat_" in c or "_syn_" in c or "diff_team_volatility" in c
) and c not in ["target", "split_key", "game_id"] and "win" not in c]

print(f"Features Totales: {len(feature_cols)}")

print("Dividiendo Train/Test...")
df = df.with_columns(pl.Series(name="split_key", values=np.random.rand(df.height)).cast(pl.Float32))


X_train = df.filter(pl.col("split_key") < 0.95).select(feature_cols).to_numpy()
y_train = df.filter(pl.col("split_key") < 0.95).select("target").to_numpy().flatten()
X_test  = df.filter(pl.col("split_key") >= 0.95).select(feature_cols).to_numpy()
y_test  = df.filter(pl.col("split_key") >= 0.95).select("target").to_numpy().flatten()

print(f"   -> Liberando memoria del DataFrame ({df.estimated_size() / 1024**2:.2f} MB)...")
del df
gc.collect()

print("Creando DMatrix (Versión Ligera)...")
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_cols)
dtest  = xgb.DMatrix(X_test, label=y_test, feature_names=feature_cols)


del X_train, y_train, X_test, y_test
gc.collect()


try:
    with open("best_hyperparameters.json", "r") as f:
        best_params = json.load(f)
    print("Usando parámetros optimizados.")
except:
    print("Usando parámetros manuales.")
    best_params = {"learning_rate": 0.008957827193783819, "max_depth": 7, "subsample": 0.5309661926071266, "colsample_bytree": 0.8548686375385406, "min_child_weight": 14, "gamma": 4.997392487608526, "lambda": 6.9850562385010155, "alpha": 0.6101433943490518}
best_params.update({
    'device': 'cuda',
    'tree_method': 'hist',
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
})

print("INICIANDO ENTRENAMIENTO...")
model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=10000,
    evals=[(dtest, "Test")],
    verbose_eval=500,
    early_stopping_rounds=500
)

print(f"Mejor Score: {model.best_score}")
model.save_model("draft_oracle_brain_v12_final.json")
with open("model_features_v12.json", "w") as f: json.dump(feature_cols, f)
print("Modelo Guardado.")